In [15]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from SALib.analyze import sobol
from ema_workbench.analysis import feature_scoring
from Convience_fun import create_scenario
import warnings
warnings.filterwarnings("ignore")

In [16]:
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    Scenario

)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time
from ema_workbench.analysis import prim
from ema_workbench import perform_experiments
from ema_workbench import Samplers
from ema_workbench import SequentialEvaluator
from ema_workbench.em_framework.parameters import Constant

import itertools

In [21]:
import multirange as mr

df_param_space_A5 = pd.read_csv('data/para_space_A5.csv').drop(0).rename(columns={"Unnamed: 0": "Variable"})
df_param_space_all = pd.read_csv('data/para_space_all.csv') .drop(0).rename(columns={"Unnamed: 0": "Variable"})

dike_model, planning_steps = get_model_for_problem_formulation(3)

def create_scenario_from_box(all, a5 ):
    para_list = []
    for uncertainty in dike_model.uncertainties:
        # print((variable in df_param_space_A5.Variable))
        variable = uncertainty.name
        # print(variable)
        if (variable in list(df_param_space_A5.Variable)) & (variable in list(df_param_space_all.Variable)) & (all + a5 ==2) :  
            min_1 = float(df_param_space_A5[df_param_space_A5['Variable'] ==variable][df_param_space_A5.columns[1]].values[0])*10000
            max_1 =float(df_param_space_A5[df_param_space_A5['Variable'] ==variable][df_param_space_A5.columns[2]].values[0])*10000
            
            min_2 = float(df_param_space_all[df_param_space_all['Variable'] ==variable][df_param_space_all.columns[1]].values[0]) *10000
            max_2 =float(df_param_space_all[df_param_space_all['Variable'] ==variable][df_param_space_all.columns[2]].values[0]) *10000
            
            boundaries = mr.overlap(range(int(min_1),int(max_1)),range(int(min_2),int(max_2)))
            para_list.append((boundaries.start + (boundaries.stop - boundaries.start)/2)/10000)  
            print((boundaries.start + (boundaries.stop - boundaries.start)/2)/10000)
        elif (variable in list(df_param_space_A5.Variable)) & (a5 == True):
            min_1 = float(df_param_space_A5[df_param_space_A5['Variable'] ==variable][df_param_space_A5.columns[1]].values[0])
            max_1 =float(df_param_space_A5[df_param_space_A5['Variable'] ==variable][df_param_space_A5.columns[2]].values[0]) 
            
            para_list.append((min_1 + (max_1 - min_1)/2))   
        elif (variable in list(df_param_space_all.Variable)) & (all == True):
            min_2 = float(df_param_space_all[df_param_space_all['Variable'] ==variable][df_param_space_all.columns[1]].values[0])
            max_2 =float(df_param_space_all[df_param_space_all['Variable'] ==variable][df_param_space_all.columns[2]].values[0])
            
            para_list.append((min_2 + (max_2 - min_2)/2))      
        elif variable.startswith("d"):
            para_list.append(3.5)
        elif variable == "A.0_ID flood wave shape":
            para_list.append(17)
        elif variable.endswith("Brate"):
            para_list.append(1.5)
        else:
            para_list.append((uncertainty.lower_bound + (uncertainty.upper_bound - uncertainty.lower_bound)/2))
    
    return create_scenario(para_list)    


Scenarios_MOEA = [create_scenario_from_box(all=True,a5=True),create_scenario_from_box(all=True,a5=False),create_scenario_from_box(all=False,a5=True)]

In [ ]:
with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(uncertainties,100)

In [ ]:
experiments, outcomes = results

In [ ]:
experiments

In [ ]:
experiments_unc = experiments.iloc[:, 19:]
outcomes.keys()
df = pd.DataFrame(outcomes)
df["Total Costs"] = df["A.1 Total Costs"] + df["A.2 Total Costs"] + df["A.3 Total Costs"] + df["A.4 Total Costs"] + df[
    "A.5 Total Costs"]
df["Expected Number of Deaths"] = df["A.1_Expected Number of Deaths"] + df["A.2_Expected Number of Deaths"] + df[
    "A.3_Expected Number of Deaths"] + df["A.4_Expected Number of Deaths"] + df["A.5_Expected Number of Deaths"]
outcomes_feaimp = outcomes
df_feature_importance_out = df[
    ["Total Costs", "Expected Number of Deaths", "A.5_Expected Number of Deaths", "A.5 Total Costs"]]
## Feature importance

x = experiments_unc
y = df_feature_importance_out
fig, ax = plt.subplots(figsize=(20,20))
fs = feature_scoring.get_feature_scores_all(x, y)
sns.heatmap(fs, cmap="viridis", annot=True,ax=ax)
plt.show()

### SOBOL

In [ ]:
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [ ]:
def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.uncertain}


uncertainties = Scenario("SCEN1",**{'discount rate 0': 0,
    'discount rate 1': 0,
    'discount rate 2': 0,
    'A.0_ID flood wave shape': 0,
    'A.1_Bmax': 200,
    'A.1_pfail': 0.9833438536152244,
    'A.1_Brate': 1 ,
    'A.2_Bmax': 200,
    'A.2_pfail': 0.9601872768253088,
    'A.2_Brate': 1,
    'A.3_Bmax': 200,
    'A.3_pfail': 0.951145688071847,
    'A.3_Brate': 1,
    'A.4_Bmax': 200,
    'A.4_pfail': 1,
    'A.4_Brate': 1,
    'A.5_Bmax': 200,
    'A.5_pfail': 0.29973752703517675,
    'A.5_Brate': 1})


In [ ]:
# with MultiprocessingEvaluator(dike_model_1) as evaluator:
#     results = evaluator.perform_experiments(scenarios=20,policies=policy0,uncertainty_sampling=Samplers.SOBOL)

In [ ]:
with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(uncertainties,10,lever_sampling=Samplers.SOBOL)

In [ ]:
experiments_unc = experiments.iloc[:, 19:]
outcomes.keys()
df = pd.DataFrame(outcomes)
df["Total Costs"] = df["A.1 Total Costs"] + df["A.2 Total Costs"] + df["A.3 Total Costs"] + df["A.4 Total Costs"] + df[
    "A.5 Total Costs"]
df["Expected Number of Deaths"] = df["A.1_Expected Number of Deaths"] + df["A.2_Expected Number of Deaths"] + df[
    "A.3_Expected Number of Deaths"] + df["A.4_Expected Number of Deaths"] + df["A.5_Expected Number of Deaths"]

df["Expected Annual Damage"] = df["A.1_Expected Annual Damage"] + df["A.2_Expected Annual Damage"] + df["A.3_Expected Annual Damage"] + df["A.4_Expected Annual Damage"] + df["A.5_Expected Annual Damage"]

df_feature_importance_out = df[
    ["Expected Annual Damage","Total Costs", "Expected Number of Deaths","A.5_Expected Annual Damage", "A.5_Expected Number of Deaths", "A.5 Total Costs"]]
## Feature importance


In [ ]:
outcomes["Total Costs"] = df["Total Costs"]
outcomes["Expected Number of Deaths"] = df["Expected Number of Deaths"]
outcomes["Expected Annual Damage"] = df["Expected Annual Damage"]

In [ ]:
problem = get_SALib_problem(dike_model.uncertainties)

sns.set_style('white')
fig, ax = plt.subplots(3, 2, figsize=(10, 10))
ax = ax.flatten()

for index, kpi in enumerate(df_feature_importance_out.columns):
    Si = sobol.analyze(problem, np.array(outcomes[kpi]), calc_second_order=True, print_to_console=False)
    Si_filter = {k: Si[k] for k in ['ST', 'ST_conf', 'S1', 'S1_conf']}
    Si_df = pd.DataFrame(Si_filter, index=problem['names'])

    indices = Si_df[['S1', 'ST']]
    err = Si_df[['S1_conf', 'ST_conf']]
    ax[index].title.set_text(kpi)
    indices.plot.bar(yerr=err.values.T, ax=ax[index])
    # fig.set_size_inches(8,6)
    # fig.subplots_adjust(bottom=0.3)
plt.tight_layout()
plt.show()